# Open AI மாடல்களை நன்றாக அமைத்தல்

இந்த நோட்புக் Open AI-இன் [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) ஆவணத்தில் வழங்கப்பட்ட தற்போதைய வழிகாட்டுதலின் அடிப்படையில் உருவாக்கப்பட்டுள்ளது.

நன்றாக அமைத்தல் (Fine-tuning) உங்கள் பயன்பாட்டிற்கான அடிப்படை மாடல்களின் செயல்திறனை மேம்படுத்துகிறது, குறிப்பிட்ட பயன்பாடு அல்லது சூழ்நிலைக்கு தொடர்புடைய கூடுதல் தரவுகள் மற்றும் சூழலுடன் மாடலை மீண்டும் பயிற்சி அளிப்பதன் மூலம். _Few shot learning_ மற்றும் _retrieval augmented generation_ போன்ற ப்ராம்ப்ட் இன்ஜினியரிங் தொழில்நுட்பங்கள், தரத்தை மேம்படுத்த தொடர்புடைய தரவுகளுடன் இயல்புநிலை ப்ராம்ப்ட்டை மேம்படுத்த அனுமதிக்கின்றன. ஆனால், இந்த அணுகுமுறைகள் இலக்கு அடிப்படை மாடலின் அதிகபட்ச டோக்கன் சாளர அளவால் கட்டுப்படுத்தப்படுகின்றன.

நன்றாக அமைத்தலின் மூலம், தேவையான தரவுகளுடன் மாடலை மீண்டும் பயிற்சி அளிக்கிறோம் (அதிகபட்ச டோக்கன் சாளரத்தில் பொருந்தக்கூடியதை விட அதிகமான உதாரணங்களை பயன்படுத்த அனுமதிக்கிறது) - மேலும், இன்ஃபரன்ஸ் நேரத்தில் உதாரணங்களை வழங்க தேவையில்லாத ஒரு _தனிப்பயன்_ பதிப்பை வெளியிடுகிறோம். இது நமது ப்ராம்ப்ட் வடிவமைப்பின் செயல்திறனை மேம்படுத்துவதோடு (டோக்கன் சாளரத்தை மற்ற விஷயங்களுக்கு பயன்படுத்துவதில் அதிக சுதந்திரம் கிடைக்கிறது) செலவுகளை குறைக்கவும் (இன்ஃபரன்ஸ் நேரத்தில் மாடலுக்கு அனுப்ப வேண்டிய டோக்கன்களின் எண்ணிக்கையை குறைப்பதன் மூலம்) உதவுகிறது.

நன்றாக அமைத்தலின் 4 படிகள்:
1. பயிற்சி தரவுகளை தயாரித்து பதிவேற்றவும்.
1. நன்றாக அமைக்கப்பட்ட மாடலை பெற பயிற்சி வேலைகளை இயக்கவும்.
1. நன்றாக அமைக்கப்பட்ட மாடலை மதிப்பீடு செய்து தரத்திற்காக திருத்தவும்.
1. திருப்தி அடைந்த பிறகு இன்ஃபரன்ஸ் பயன்பாட்டிற்கு மாடலை வெளியிடவும்.

அனைத்து அடிப்படை மாடல்களும் நன்றாக அமைத்தலை ஆதரிக்காது என்பதை கவனிக்கவும் - [OpenAI ஆவணங்களை](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) சமீபத்திய தகவலுக்காக சரிபார்க்கவும். நீங்கள் ஏற்கனவே நன்றாக அமைக்கப்பட்ட மாடலை மீண்டும் நன்றாக அமைக்கலாம். இந்த டுடோரியலில், `gpt-35-turbo` மாடலை நன்றாக அமைக்க இலக்கு அடிப்படை மாடலாக பயன்படுத்துவோம்.

---


### படி 1.1: உங்கள் தரவுத்தொகுப்பை தயாரிக்கவும்

நாம் ஒரு சாட்பாட்டை உருவாக்கப் போகிறோம், இது உங்களுக்கு மூலக்கூறுகளின் காலக்கட்ட அட்டவணையைப் புரிந்துகொள்ள உதவுகிறது, ஒரு மூலக்கூறைப் பற்றிய கேள்விகளுக்கு ஒரு லிமரிக் மூலம் பதிலளிக்கிறது. _இந்த_ எளிய பயிற்சியில், தரவின் எதிர்பார்க்கப்படும் வடிவத்தை காட்டும் சில மாதிரி பதில்களின் உதாரணங்களுடன் மாதிரி உருவாக்குவதற்கான ஒரு தரவுத்தொகுப்பை உருவாக்குவோம். ஒரு உண்மையான பயன்பாட்டில், நீங்கள் மேலும் பல உதாரணங்களுடன் ஒரு தரவுத்தொகுப்பை உருவாக்க வேண்டும். உங்கள் பயன்பாட்டு துறைக்கு ஏற்ற ஒரு திறந்த தரவுத்தொகுப்பு இருந்தால், அதை மீள வடிவமைத்து நுண்ணிய-சீரமைப்பிற்குப் பயன்படுத்தவும் முடியும்.

நாம் `gpt-35-turbo`-ஐ மையமாகக் கொண்டு, ஒற்றை-முறை பதில்களை (சாட் நிறைவு) எதிர்பார்க்கிறோம், எனவே [இந்த பரிந்துரைக்கப்பட்ட வடிவத்தை](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) பயன்படுத்தி OpenAI சாட் நிறைவு தேவைகளை பிரதிபலிக்கும் உதாரணங்களை உருவாக்கலாம். நீங்கள் பல-முறை உரையாடல் உள்ளடக்கத்தை எதிர்பார்க்கிறீர்கள் என்றால், [பல-முறை உதாரண வடிவத்தை](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) பயன்படுத்த வேண்டும், இது `weight` அளவுருவைச் சேர்க்கிறது, எந்த செய்திகளை நுண்ணிய-சீரமைப்பு செயல்முறையில் பயன்படுத்த வேண்டும் (அல்லது வேண்டாம்) என்பதை சுட்டிக்காட்ட.

இந்த பயிற்சிக்காக எளிய ஒற்றை-முறை வடிவத்தைப் பயன்படுத்துவோம். தரவுகள் [jsonl வடிவத்தில்](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) இருக்கும், ஒவ்வொரு வரியிலும் 1 பதிவுடன், ஒவ்வொன்றும் JSON வடிவமைக்கப்பட்ட பொருளாக பிரதிநிதித்துவம் செய்யப்படும். கீழே உள்ள குறுக்கம் 2 பதிவுகளை மாதிரியாகக் காட்டுகிறது - முழு மாதிரி தொகுப்புக்கான [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) ஐப் பார்க்கவும் (10 உதாரணங்கள்) நமது நுண்ணிய-சீரமைப்பு பயிற்சிக்காக பயன்படுத்துவோம். **குறிப்பு:** ஒவ்வொரு பதிவும் _ஒரு_ வரியில் வரையறுக்கப்பட வேண்டும் (சாதாரண JSON கோப்பில் உள்ளதைப் போல வரிகளில் பிரிக்கப்படக்கூடாது).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

உண்மையான பயன்பாட்டில், நல்ல முடிவுகளுக்காக மிகப் பெரிய உதாரணத் தொகுப்பை உருவாக்க வேண்டும் - பதில்களின் தரம் மற்றும் நுண்ணிய-சீரமைப்பிற்கான நேரம்/செலவுகள் ஆகியவற்றுக்கு இடையிலான சமநிலை இருக்கும். நாங்கள் ஒரு சிறிய தொகுப்பைப் பயன்படுத்துகிறோம், நுண்ணிய-சீரமைப்பு செயல்முறையை விரைவாக முடிக்க, செயல்முறையை விளக்க. மேலும் சிக்கலான நுண்ணிய-சீரமைப்பு பயிற்சிக்கான [இந்த OpenAI Cookbook உதாரணத்தை](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) பார்க்கவும்.


---

### படி 1.2 உங்கள் தரவுத்தொகுப்பை பதிவேற்றவும்

Files API-யைப் பயன்படுத்தி [இங்கு விவரிக்கப்பட்டது](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) போல தரவுகளை பதிவேற்றவும். இந்தக் குறியீட்டை இயக்குவதற்கு முன், நீங்கள் பின்வரும் படிகளை முடித்திருக்க வேண்டும்:
 - `openai` Python தொகுப்பை நிறுவியிருக்க வேண்டும் (சமீபத்திய அம்சங்களுக்காக >=0.28.0 பதிப்பைப் பயன்படுத்தவும்)
 - உங்கள் OpenAI API விசையை `OPENAI_API_KEY` சூழல் மாறியாக அமைத்திருக்க வேண்டும்
மேலும் அறிய, இந்த பாடத்திற்கான [Setup guide](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) ஐப் பார்க்கவும்.

இப்போது, உங்கள் உள்ளூர் JSONL கோப்பிலிருந்து பதிவேற்றத்திற்கான கோப்பை உருவாக்க குறியீட்டை இயக்கவும்.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### படி 2.1: SDK மூலம் Fine-tuning வேலை உருவாக்கவும்


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### படி 2.2: வேலைக்கான நிலையை சரிபார்க்கவும்

`client.fine_tuning.jobs` API மூலம் நீங்கள் செய்யக்கூடிய சில செயல்பாடுகள் இவை:
- `client.fine_tuning.jobs.list(limit=<n>)` - கடைசி n நுண்ணிய துனை வேலைகளை பட்டியலிடுங்கள்
- `client.fine_tuning.jobs.retrieve(<job_id>)` - ஒரு குறிப்பிட்ட நுண்ணிய துனை வேலை பற்றிய விவரங்களை பெறுங்கள்
- `client.fine_tuning.jobs.cancel(<job_id>)` - ஒரு நுண்ணிய துனை வேலை ரத்து செய்யுங்கள்
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - வேலைக்கான n நிகழ்வுகளை பட்டியலிடுங்கள்
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

இந்த செயல்முறையின் முதல் படி _பயிற்சி கோப்பை சரிபார்ப்பது_ ஆகும், அதாவது தரவுகள் சரியான வடிவத்தில் உள்ளதா என்பதை உறுதிப்படுத்துவது.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### படி 2.3: முன்னேற்றத்தை கண்காணிக்க நிகழ்வுகளைப் பதிவு செய்யவும்


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### படி 2.4: OpenAI டாஷ்போர்டில் நிலையைப் பார்வையிடவும்


நீங்கள் OpenAI வலைத்தளத்தை பார்வையிட்டு, _Fine-tuning_ பிரிவை ஆராய்ந்து நிலையைப் பார்க்கலாம். இது தற்போதைய வேலை நிலையை காட்டும், மேலும் முந்தைய வேலை செயல்பாடுகளின் வரலாற்றையும் கண்காணிக்க அனுமதிக்கும். இந்த ஸ்கிரீன்ஷாட்டில், முந்தைய செயல்பாடு தோல்வியடைந்தது, மற்றும் இரண்டாவது செயல்பாடு வெற்றிகரமாக முடிந்தது என்பதை நீங்கள் காணலாம். விளக்கமாக, முதல் செயல்பாட்டில் தவறாக வடிவமைக்கப்பட்ட பதிவுகளுடன் JSON கோப்பு பயன்படுத்தப்பட்டது - அதை சரிசெய்த பிறகு, இரண்டாவது செயல்பாடு வெற்றிகரமாக முடிந்து, மாடலைப் பயன்படுத்த தயாராக வைத்தது.

![Fine-tuning வேலை நிலை](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.ta.png)


நீங்கள் மேலும் கீழே சென்று காட்சிப்படுத்தப்பட்ட டாஷ்போர்டில் நிலைச் செய்திகளையும் அளவுகோள்களையும் காணலாம்:

| செய்திகள் | அளவுகோல்கள் |
|:---|:---|
| ![செய்திகள்](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.ta.png) |  ![அளவுகோல்கள்](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.ta.png)|


---

### படி 3.1: ID ஐ பெறுதல் & கோடில் நன்றாகத் திருத்தப்பட்ட மாடலை சோதித்தல்


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### படி 3.2: நன்றாகத் தகுந்த மாடலை Playground-ல் ஏற்றவும் மற்றும் சோதிக்கவும்

நீங்கள் இப்போது நன்றாகத் தகுந்த மாடலை இரண்டு வழிகளில் சோதிக்கலாம். முதலில், நீங்கள் Playground-க்கு சென்று Models drop-down-ஐ பயன்படுத்தி உங்கள் புதிய நன்றாகத் தகுந்த மாடலை பட்டியலில் இருந்து தேர்ந்தெடுக்கலாம். மற்றொரு விருப்பம் Fine-tuning panel-ல் காணப்படும் "Playground" விருப்பத்தை பயன்படுத்துவது (மேலே உள்ள ஸ்கிரீன்ஷாட்டைப் பார்க்கவும்), இது கீழே உள்ள _ஒப்பீட்டு_ காட்சியை தொடங்குகிறது, இது அடிப்படை மற்றும் நன்றாகத் தகுந்த மாடல் பதிப்புகளை பக்கத்தில் பக்கமாக விரைவான மதிப்பீட்டிற்காகக் காட்டுகிறது.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.ta.png)

உங்கள் பயிற்சி தரவுகளில் பயன்படுத்திய system context-ஐ நிரப்பி உங்கள் சோதனை கேள்வியை வழங்கவும். இரு பக்கங்களும் ஒரே context மற்றும் கேள்வியுடன் புதுப்பிக்கப்படும் என்பதை நீங்கள் கவனிக்கலாம். ஒப்பீட்டை இயக்கி, அவற்றின் வெளியீடுகளில் உள்ள வேறுபாட்டை நீங்கள் காணலாம். _உங்கள் உதாரணங்களில் நீங்கள் வழங்கிய வடிவத்தில் நன்றாகத் தகுந்த மாடல் பதிலை உருவாக்கும் விதத்தை கவனிக்கவும், ஆனால் அடிப்படை மாடல் system prompt-ஐ மட்டும் பின்பற்றுகிறது_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.ta.png)

நீங்கள் கவனிக்கக்கூடியது, ஒப்பீட்டில் ஒவ்வொரு மாடலுக்கும் token எண்ணிக்கைகள் மற்றும் inference-க்கு எடுத்த நேரம் ஆகியவை வழங்கப்படும். **இந்த குறிப்பிட்ட உதாரணம் செயல்முறையை காட்டுவதற்காக எளிமையானது, ஆனால் உண்மையான உலக தரவுத்தொகுப்பு அல்லது சூழ்நிலையை பிரதிபலிக்கவில்லை**. நீங்கள் கவனிக்கலாம், இரண்டு மாதிரிகளும் ஒரே token எண்ணிக்கையை (system context மற்றும் user prompt ஒரே மாதிரியானவை) காட்டுகின்றன, ஆனால் நன்றாகத் தகுந்த மாடல் inference-க்கு அதிக நேரம் எடுத்துக்கொள்கிறது (custom model).

உண்மையான உலக சூழ்நிலைகளில், நீங்கள் இவ்வாறு ஒரு எளிய உதாரணத்தை பயன்படுத்தமாட்டீர்கள், ஆனால் உண்மையான தரவுகளுக்கு (எ.கா., வாடிக்கையாளர் சேவைக்கான தயாரிப்பு பட்டியல்) எதிராக நன்றாகத் தகுந்த மாடலை fine-tune செய்வீர்கள், அங்கு பதிலின் தரம் மிகவும் தெளிவாக இருக்கும். அந்த சூழலில், foundation model-க்கு சமமான பதிலின் தரத்தைப் பெற, அதிக custom prompt engineering தேவைப்படும், இது token பயன்பாட்டை அதிகரிக்கும் மற்றும் inference-க்கு தொடர்புடைய செயலாக்க நேரத்தை அதிகரிக்கும். _இதனை முயற்சிக்க, OpenAI Cookbook-ல் fine-tuning உதாரணங்களைப் பாருங்கள்._

---



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
